# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [67]:
# Dependencies and Setup
import hvplot.pandas
import warnings
import pandas as pd
import requests
import holoviews
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [68]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kapaa,22.0752,-159.3190,26.18,81,20,1.54,US,1673929450
1,1,bucerias,20.7500,-105.3333,18.99,82,0,1.03,MX,1673929451
2,2,bengkulu,-3.8004,102.2655,28.33,67,66,2.68,ID,1673929452
3,3,ratnagiri,16.9833,73.3000,23.16,61,4,1.90,IN,1673929454
4,4,margate,51.3813,1.3862,-2.05,66,98,2.58,GB,1673929455


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [78]:
# Configure the map plot
city_data_map = city_data_df.hvplot.points('Lng','Lat',geo = True, tiles = True,frame_width = 800,frame_height = 600,size = 'Humidity',c = 'City')
# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [86]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 27 degrees but higher than 21
ideal_city_df = city_data_df[city_data_df['Max Temp']>=21]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp']<=27]
# Wind speed less than 4.5 m/s
ideal_city_df = ideal_city_df[ideal_city_df['Wind Speed']<4.5]
# Zero cloudiness
ideal_city_df = ideal_city_df[ideal_city_df['Cloudiness']==0]
# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,busselton,-33.6500,115.3333,22.49,29,0,2.29,AU,1673929472
55,55,cape town,-33.9258,18.4232,21.88,79,0,3.09,ZA,1673929530
241,241,ewa beach,21.3156,-158.0072,26.97,65,0,4.12,US,1673929786
250,250,makakilo city,21.3469,-158.0858,26.18,65,0,4.12,US,1673929800
299,299,pisco,-13.7000,-76.2167,21.12,78,0,0.00,PE,1673929887


### Step 3: Create a new DataFrame called `hotel_df`.

In [89]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,busselton,-33.6500,115.3333,22.49,29,0,2.29,AU,1673929472,
55,55,cape town,-33.9258,18.4232,21.88,79,0,3.09,ZA,1673929530,
241,241,ewa beach,21.3156,-158.0072,26.97,65,0,4.12,US,1673929786,
250,250,makakilo city,21.3469,-158.0858,26.18,65,0,4.12,US,1673929800,
299,299,pisco,-13.7000,-76.2167,21.12,78,0,0.00,PE,1673929887,
315,315,rio cuarto,-33.1307,-64.3499,22.95,60,0,1.54,AR,1673929808,
324,324,maraba,-5.3686,-49.1178,24.10,100,0,0.00,BR,1673929922,
389,389,mohgaon,21.6500,78.7167,23.32,39,0,1.99,IN,1673930019,
515,515,cordoba,-31.4135,-64.1811,23.99,56,0,3.09,AR,1673930194,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [102]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":'accommodation',
    'limit':1,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # Make and API request using the params dictionaty and convert to JSON
    name_address = requests.get(base_url,params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: Gale Street Motel & Villas
cape town - nearest hotel: Townhouse Hotel
ewa beach - nearest hotel: Hampton Inn & Suites Oahu/Kapolei
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
pisco - nearest hotel: No hotel found
rio cuarto - nearest hotel: HotelSanCarlos
maraba - nearest hotel: Portobelo
mohgaon - nearest hotel: No hotel found
cordoba - nearest hotel: alvear


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,busselton,-33.6500,115.3333,22.49,29,0,2.29,AU,1673929472,Gale Street Motel & Villas
55,55,cape town,-33.9258,18.4232,21.88,79,0,3.09,ZA,1673929530,Townhouse Hotel
241,241,ewa beach,21.3156,-158.0072,26.97,65,0,4.12,US,1673929786,Hampton Inn & Suites Oahu/Kapolei
250,250,makakilo city,21.3469,-158.0858,26.18,65,0,4.12,US,1673929800,Embassy Suites by Hilton Oahu Kapolei
299,299,pisco,-13.7000,-76.2167,21.12,78,0,0.00,PE,1673929887,No hotel found
315,315,rio cuarto,-33.1307,-64.3499,22.95,60,0,1.54,AR,1673929808,HotelSanCarlos
324,324,maraba,-5.3686,-49.1178,24.10,100,0,0.00,BR,1673929922,Portobelo
389,389,mohgaon,21.6500,78.7167,23.32,39,0,1.99,IN,1673930019,No hotel found
515,515,cordoba,-31.4135,-64.1811,23.99,56,0,3.09,AR,1673930194,alvear


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [105]:
# Configure the map plot
hotel_df_map = hotel_df.hvplot.points('Lng','Lat',geo = True, tiles = True,frame_width = 800,frame_height = 600,size = 'Humidity',c = 'City',hover_cols=['Hotel Name', 'Country'])
# Display the map
hotel_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)